In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Obesity"
cohort = "GSE123086"

# Input paths
in_trait_dir = "../../input/GEO/Obesity"
in_cohort_dir = "../../input/GEO/Obesity/GSE123086"

# Output paths
out_data_file = "../../output/preprocess/Obesity/GSE123086.csv"
out_gene_data_file = "../../output/preprocess/Obesity/gene_data/GSE123086.csv"
out_clinical_data_file = "../../output/preprocess/Obesity/clinical_data/GSE123086.csv"
json_path = "../../output/preprocess/Obesity/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"A validated single-cell-based strategy to identify diagnostic and therapeutic targets in complex diseases [study of 13 diseases]"
!Series_summary	"We conducted prospective clinical studies to validate the importance of CD4+ T cells in 13 diseases from the following ICD-10-CM chapters: Neoplasms (breast cancer, chronic lymphocytic leukemia); endocrine, nutritional and metabolic diseases (type I diabetes, obesity); diseases of the circulatory system (atherosclerosis); diseases of the respiratory system (acute tonsillitis, influenza, seasonal allergic rhinitis, asthma); diseases of the digestive system (Crohn’s disease [CD], ulcerative colitis [UC]); and diseases of the skin and subcutaneous tissue (atopic eczema, psoriatic diseases)."
!Series_summary	"Study participants were recruited by clinical specialists based on diagnostic criteria defined by organizations representing each specialist’s discipline. Age and gender matched healthy controls (n = 1

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# Analyze the dataset

# 1. Gene Expression Data Availability
# Based on the background information, this dataset contains gene expression data from microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Obesity)
# From the sample characteristics, primary diagnosis is in row 1, and it includes 'OBESITY'
trait_row = 1

# For age
# Age information appears to be in rows 3 and 4
age_row = 3  # Primary age information

# For gender
# Gender (Sex) information appears in rows 2 and 3
gender_row = 2  # Primary sex information

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert trait value to binary (0 or 1)."""
    if pd.isna(value):
        return None
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if the value indicates obesity
    if "OBESITY" in value:
        return 1
    elif "HEALTHY_CONTROL" in value:
        return 0
    else:
        return None  # Not related to obesity or control

def convert_age(value):
    """Convert age value to continuous numeric value."""
    if pd.isna(value):
        return None
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Try to convert to float
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)."""
    if pd.isna(value):
        return None
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == "MALE":
        return 1
    elif value.upper() == "FEMALE":
        return 0
    else:
        # Not a gender value (might be a diagnosis entry in the same row)
        return None

# 3. Save Metadata - Initial Filtering
# Determine if trait data is available
is_trait_available = trait_row is not None

# Validate and save cohort information
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (if trait data is available)
if trait_row is not None:
    # Create a DataFrame from the sample characteristics dictionary provided in the previous output
    # First, load or create the sample characteristics data
    sample_chars_dict = {
        0: ['cell type: CD4+ T cells'], 
        1: ['primary diagnosis: ASTHMA', 'primary diagnosis: ATHEROSCLEROSIS', 'primary diagnosis: BREAST_CANCER', 
            'primary diagnosis: CHRONIC_LYMPHOCYTIC_LEUKEMIA', 'primary diagnosis: CROHN_DISEASE', 
            'primary diagnosis: ATOPIC_ECZEMA', 'primary diagnosis: HEALTHY_CONTROL', 'primary diagnosis: INFLUENZA', 
            'primary diagnosis: OBESITY', 'primary diagnosis: PSORIASIS', 
            'primary diagnosis: SEASONAL_ALLERGIC_RHINITIS', 'primary diagnosis: TYPE_1_DIABETES', 
            'primary diagnosis: ACUTE_TONSILLITIS', 'primary diagnosis: ULCERATIVE_COLITIS'], 
        2: ['Sex: Male', 'diagnosis2: ATOPIC_ECZEMA', 'Sex: Female', 'diagnosis2: ATHEROSCLEROSIS', 
            'diagnosis2: ASTHMA_OBESITY', 'diagnosis2: ASTHMA', 'diagnosis2: ASTMHA_SEASONAL_ALLERGIC_RHINITIS', 
            'diagnosis2: OBESITY'], 
        3: ['age: 56', 'Sex: Male', 'age: 20', 'age: 51', 'age: 37', 'age: 61', 'age: 31', 'age: 41', 'age: 80', 
            'age: 53', 'age: 73', 'age: 60', 'age: 76', 'age: 77', 'age: 74', 'age: 69', 'age: 81', 'age: 70', 
            'age: 82', 'age: 67', 'age: 78', 'age: 72', 'age: 66', 'age: 36', 'age: 45', 'age: 65', 'age: 48', 
            'age: 50', 'age: 24', 'age: 42'], 
        4: [pd.NA, 'age: 63', 'age: 74', 'age: 49', 'age: 60', 'age: 68', 'age: 38', 'age: 16', 'age: 12', 'age: 27']
    }
    
    # Convert the dictionary to a DataFrame that can be processed by geo_select_clinical_features
    # This is a simplified approach - we need to transpose the data to have samples as rows
    # First, create a list of all unique sample IDs
    sample_ids = []
    for values in sample_chars_dict.values():
        for value in values:
            if not pd.isna(value):
                sample_ids.append(f"Sample_{len(sample_ids)}")
    
    # Create a DataFrame with sample IDs as index
    clinical_data = pd.DataFrame(index=sample_ids)
    
    # Add each characteristic as a column
    for row_idx, values in sample_chars_dict.items():
        for i, value in enumerate(values):
            if i < len(sample_ids) and not pd.isna(value):
                clinical_data.loc[sample_ids[i], f"Characteristic_{row_idx}"] = value
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the selected clinical features
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Save the clinical data to CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'Characteristic_0': [nan, nan, nan], 'Characteristic_1': [nan, nan, nan], 'Characteristic_2': [nan, nan, 0.0], 'Characteristic_3': [nan, 51.0, nan], 'Characteristic_4': [nan, 49.0, nan]}
Clinical data saved to ../../output/preprocess/Obesity/clinical_data/GSE123086.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


First 20 gene/probe identifiers:
Index(['1', '2', '3', '9', '10', '12', '13', '14', '15', '16', '18', '19',
       '20', '21', '22', '23', '24', '25', '26', '27'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be numerical IDs or probe identifiers, not standard human gene symbols.
# Standard human gene symbols are typically alphanumeric (like BRCA1, TP53, IL6) rather than simple 
# sequential numbers like 1, 2, 3, etc.
# Therefore, these identifiers will need to be mapped to proper gene symbols for meaningful analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# Let's try to fully examine the SOFT file to find the gene symbol information
import gzip
import re

# Function to search for platform table data in the SOFT file
def extract_platform_data(file_path):
    inside_platform_table = False
    platform_data = []
    
    with gzip.open(file_path, 'rt') as f:
        for line in f:
            line = line.strip()
            if line == "!Platform_table_begin":
                inside_platform_table = True
                continue
            elif line == "!Platform_table_end":
                inside_platform_table = False
                break
            
            if inside_platform_table:
                platform_data.append(line)
    
    return platform_data

# Let's try to extract the platform data manually
platform_data = extract_platform_data(soft_file)

if platform_data:
    print("Found platform data! First few lines:")
    for line in platform_data[:5]:
        print(line)
    
    # Check if there's a header line
    if platform_data[0].startswith("ID"):
        header = platform_data[0].split('\t')
        print("\nHeader fields:")
        print(header)
        
        # Look for gene symbol or related columns
        gene_symbol_columns = [i for i, col in enumerate(header) if 'SYMBOL' in col.upper() or 'GENE' in col.upper()]
        if gene_symbol_columns:
            print("\nPotential gene symbol columns found at indices:")
            for idx in gene_symbol_columns:
                print(f"{idx}: {header[idx]}")
else:
    print("No platform data found using direct extraction method.")

# As a fallback, let's try the GPL ID approach
# First, find the GPL ID in the SOFT file
gpl_id = None
with gzip.open(soft_file, 'rt') as f:
    for line in f:
        if line.startswith("!Series_platform_id"):
            gpl_id = line.strip().split("=")[1].strip()
            break

if gpl_id:
    print(f"\nFound platform ID: {gpl_id}")
    
    # Check if it matches standard Agilent arrays which are commonly used
    if re.match(r'GPL\d+', gpl_id):
        print(f"This is a standard GEO platform: {gpl_id}")
        print("For this platform, ENTREZ_GENE_ID can be used to map to gene symbols.")
        print("We'll use the ENTREZ_GENE_ID for mapping in the next step.")

# Our conclusion about the gene annotation data
print("\nBased on the available information:")
print("1. The gene annotation contains ENTREZ_GENE_ID which can be used to map to gene symbols")
print("2. The annotation data from the SOFT file does not directly contain gene symbols")
print("3. We will need to use the Entrez Gene IDs to map to official gene symbols in the next step")

# For now, let's use what we have - the annotation with Entrez Gene IDs
gene_annotation = get_gene_annotation(soft_file)
print("\nFinal gene annotation preview:")
print(preview_df(gene_annotation))


No platform data found using direct extraction method.

Found platform ID: GPL25864
This is a standard GEO platform: GPL25864
For this platform, ENTREZ_GENE_ID can be used to map to gene symbols.
We'll use the ENTREZ_GENE_ID for mapping in the next step.

Based on the available information:
1. The gene annotation contains ENTREZ_GENE_ID which can be used to map to gene symbols
2. The annotation data from the SOFT file does not directly contain gene symbols
3. We will need to use the Entrez Gene IDs to map to official gene symbols in the next step



Final gene annotation preview:
{'ID': ['1', '2', '3', '9', '10'], 'ENTREZ_GENE_ID': ['1', '2', '3', '9', '10'], 'SPOT_ID': [1.0, 2.0, 3.0, 9.0, 10.0]}


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the previous output, we can see that:
# - The gene expression data uses numeric IDs (1, 2, 3, etc.)
# - The gene annotation has matching IDs in the 'ID' column
# - The matching 'ENTREZ_GENE_ID' column contains Entrez Gene IDs which represent actual genes

# Set up the column names for mapping
# The ID in gene_data (probe identifier) matches the 'ID' column in the annotation
prob_col = 'ID'
# The gene symbol/identifier we want to map to is in the 'ENTREZ_GENE_ID' column
gene_col = 'ENTREZ_GENE_ID'

# 2. Get the gene mapping dataframe by extracting ID and ENTREZ_GENE_ID columns
# We need to reset the index of gene_annotation to access the ID column
gene_annotation = gene_annotation.reset_index(drop=True)
mapping_df = get_gene_mapping(gene_annotation, prob_col, gene_col)

print("First 5 rows of the mapping dataframe:")
print(mapping_df.head())

# 3. Convert probe-level measurements to gene-level expression by applying the mapping
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Verify the conversion worked by showing the first few rows
print("\nFirst 5 rows of gene expression data after mapping:")
print(gene_data.head())

# Let's also check how many unique genes we have after mapping
print(f"\nNumber of unique genes after mapping: {len(gene_data)}")

# Save the gene data to csv
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

First 5 rows of the mapping dataframe:
   ID Gene
0   1    1
1   2    2
2   3    3
3   9    9
4  10   10



First 5 rows of gene expression data after mapping:
Empty DataFrame
Columns: [GSM3494884, GSM3494885, GSM3494886, GSM3494887, GSM3494888, GSM3494889, GSM3494890, GSM3494891, GSM3494892, GSM3494893, GSM3494894, GSM3494895, GSM3494896, GSM3494897, GSM3494898, GSM3494899, GSM3494900, GSM3494901, GSM3494902, GSM3494903, GSM3494904, GSM3494905, GSM3494906, GSM3494907, GSM3494908, GSM3494909, GSM3494910, GSM3494911, GSM3494912, GSM3494913, GSM3494914, GSM3494915, GSM3494916, GSM3494917, GSM3494918, GSM3494919, GSM3494920, GSM3494921, GSM3494922, GSM3494923, GSM3494924, GSM3494925, GSM3494926, GSM3494927, GSM3494928, GSM3494929, GSM3494930, GSM3494931, GSM3494932, GSM3494933, GSM3494934, GSM3494935, GSM3494936, GSM3494937, GSM3494938, GSM3494939, GSM3494940, GSM3494941, GSM3494942, GSM3494943, GSM3494944, GSM3494945, GSM3494946, GSM3494947, GSM3494948, GSM3494949, GSM3494950, GSM3494951, GSM3494952, GSM3494953, GSM3494954, GSM3494955, GSM3494956, GSM3494957, GSM3494958, GSM3494959, GSM349496